## Importing Libraries

In [227]:
import os
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
import warnings
import psycopg2
from psycopg2 import Error
from statsmodels.sandbox.stats.runs import runstest_1samp
from sqlalchemy import create_engine
from sqlalchemy import text
warnings.filterwarnings("ignore")

## Importing Data From DB

In [202]:
process_start=dt.datetime.now()

In [2]:
connection = psycopg2.connect(user="",
                                  password="",
                                  host="",
                                  port="",
                                  database="")
    
df=pd.read_sql('''select date(seg_start) as date_col, seg_start as datetime_col, idnt_unique_appel as callid,
                    libl_activite as liblactivite_vaca, num_appel_qualifie as customerid,
                    coalesce(cds_modelling_arpu,0) as proxy_metric_new,
                    case when benchmark=1 then 1 else 0 end  as on_off, 
                    case when (coalesce(cds_modelling_arpu,0))> 0 then 1 else 0 end as issale
                    from schema.table
                    where libl_activite in ('NECTAR', 'NUM COURTS', 'C2C') 
                    and disp_vdn in (2000436,2000437,2000438,2000440,2000441,2000442,2000432,2000433,
                    2000416,2000417,2000418,2000419,2000420,2000421,2000422)
                    and (num_appel_qualifie ~* '^\d+?$') is true 
                    and num_appel_qualifie not in ('0171025800','0187250000','0187037000', '0000000000')
                    and coalesce(cds_modelling_arpu,0) >= 0
                    and seg_start >='2021-03-01' order by seg_start asc;''', connection)
print("Table has been fetched:\n\nShape is: ", df.shape)
df.head()

Table has been fetched:

Shape is:  (1353817, 8)


,date_col,datetime_col,callid,liblactivite_vaca,customerid,proxy_metric_new,on_off,issale
0,2021-03-01,2021-03-01 11:54:14,00001032431614596054,NECTAR,0130795172,239.19,0,1
1,2021-03-02,2021-03-02 09:00:31,00001077051614672031,NUM COURTS,0442317256,0.00,0,0
2,2021-03-02,2021-03-02 09:00:58,00001084751614672058,NUM COURTS,0160201115,0.00,0,0
3,2021-03-02,2021-03-02 09:01:06,00001087231614672066,NUM COURTS,0698701414,0.00,0,0
4,2021-03-02,2021-03-02 09:01:08,00001087901614672068,NUM COURTS,0618915588,0.00,0,0


In [4]:
df["date_col"]= pd.to_datetime(df["date_col"])
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
df.head()

,date_col,datetime_col,callid,liblactivite_vaca,customerid,proxy_metric_new,on_off,issale
0,2021-03-01,2021-03-01 11:54:14,00001032431614596054,NECTAR,0130795172,239.19,0,1
1,2021-03-02,2021-03-02 09:00:31,00001077051614672031,NUM COURTS,0442317256,0.00,0,0
2,2021-03-02,2021-03-02 09:00:58,00001084751614672058,NUM COURTS,0160201115,0.00,0,0
3,2021-03-02,2021-03-02 09:01:06,00001087231614672066,NUM COURTS,0698701414,0.00,0,0
4,2021-03-02,2021-03-02 09:01:08,00001087901614672068,NUM COURTS,0618915588,0.00,0,0


# DESCRIPTIVE ANALYSIS

#### In this lab we will be covering topics of the following order

1) Basic Infromation regarding dataset

2) Null Value Analysis

3) Univariate Analysis using Pandas Profiling

4) Month Wise Statistics

5) Country Wise Statistics 

6) Customer Wise Analysis using RFM Quantitative Technique

### 1) Basic Infromation regarding dataset

In [5]:
df.shape

(1353817, 8)

In [6]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1353817 entries, 0 to 1353816
Data columns (total 8 columns):
date_col             1353817 non-null datetime64[ns]
datetime_col         1353817 non-null datetime64[ns]
callid               1353817 non-null object
liblactivite_vaca    1353817 non-null object
customerid           1353817 non-null object
proxy_metric_new     1353817 non-null float64
on_off               1353817 non-null int64
issale               1353817 non-null int64
dtypes: datetime64[ns](2), float64(1), int64(2), object(3)
memory usage: 82.6+ MB


In [7]:
df.describe()

,proxy_metric_new,on_off,issale
count,1.353817e+06,1.353817e+06,1.353817e+06
mean,2.378704e+01,2.408073e-01,1.692415e-01
std,6.698688e+01,4.275737e-01,3.749652e-01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00
max,1.717640e+03,1.000000e+00,1.000000e+00


In [8]:
df.describe(include= "object")

,callid,liblactivite_vaca,customerid
count,1353817,1353817,1353817
unique,1353816,3,897502
top,00001034171625560385,NUM COURTS,0187649431
freq,2,726369,2342


### 2) Null Value Analysis

In [10]:
df.isnull().sum()

date_col             0
datetime_col         0
callid               0
liblactivite_vaca    0
customerid           0
proxy_metric_new     0
on_off               0
issale               0
dtype: int64

In [11]:
cnt=0
for row in df['customerid']:
    if(len(str(row))!=10) or (any(c.isalpha()for c in row)):
        df.loc[cnt,'customerid']=np.nan
    cnt+=1
    
df=df[pd.notnull(df['customerid'])]    #we can also use dropna function as well
df=df[(df['proxy_metric_new']>=0)]

In [14]:
df.isnull().sum()

date_col             0
datetime_col         0
callid               0
liblactivite_vaca    0
customerid           0
proxy_metric_new     0
on_off               0
issale               0
dtype: int64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1353817 entries, 0 to 1353816
Data columns (total 8 columns):
date_col             1353817 non-null datetime64[ns]
datetime_col         1353817 non-null datetime64[ns]
callid               1353817 non-null object
liblactivite_vaca    1353817 non-null object
customerid           1353817 non-null object
proxy_metric_new     1353817 non-null float64
on_off               1353817 non-null int64
issale               1353817 non-null int64
dtypes: datetime64[ns](2), float64(1), int64(2), object(3)
memory usage: 93.0+ MB


### 4) Month Wise Statistics

In [16]:
df["Month"]=df["date_col"].dt.month

In [17]:
df.groupby(
    ['Month']
).agg(
    {
        'proxy_metric_new': ["mean", sum],
        'callid': "nunique", 
        'customerid': "nunique",
    }
)

proxy_metric_new              callid customerid
                  mean         sum nunique    nunique
Month                                                
1            38.458459   719903.89   18719      15598
3            21.472359  2388391.93  111231      74357
4            23.431274  2938539.46  125411      84103
5            22.802424  2680630.17  117559      82208
6            18.890562  2432896.56  128789      94635
7            21.171178  2670214.81  126124      95482
8            20.925945  3270160.19  156273     114011
9            22.235520  3490153.99  156963     118375
10           25.594453  3501833.05  136820     103204
11           25.938452  3435107.07  132433     103435
12           32.583029  4675469.11  143494     109653

### 5) Activity Wise Statistics

In [18]:
df.groupby(
    ['liblactivite_vaca']
).agg(
    {
       'proxy_metric_new': ["mean", sum],
        'callid': "nunique", 
        'customerid': "nunique",
    }
)

proxy_metric_new                callid customerid
                              mean           sum nunique    nunique
liblactivite_vaca                                                  
C2C                      27.822830  9.644673e+06  346645     243963
NECTAR                   24.402735  6.852337e+06  280802     232598
NUM COURTS               21.623019  1.570629e+07  726369     471119

## 6) Customer Wise Analysis using RFM Quantitative Technique

#### This part will analyze the behaviour of customer and catagorize each customer to be paid attention furtherly.

*  To define recency, we need to find the range of time from latest purchase to each purchase time on the dataset.
*  After that, we define frequent by summing up ordered items of each customer
*  To define monetary we sum up ordered items values of each customer.

In [201]:
max_date = max(df.date_col) + dt.timedelta(days=1)
print(max_date)

2022-01-06 00:00:00


In [204]:
rfm_start_time=dt.datetime.now()

In [205]:
RFM_data = df.groupby(['customerid']).agg({'date_col': lambda x: (max_date - x.max()).days,
                                           'callid': 'nunique',
                                           'proxy_metric_new': 'sum'})

RFM_data.rename(columns = {'date_col': 'Recency',
                           'callid': 'Frequency',
                           'proxy_metric_new': 'MonetaryValue'},
                inplace=True)
RFM_data.head()

,Recency,Frequency,MonetaryValue
customerid,,,
0003503118,234,3,0.00
0014521825,159,1,0.00
0022804213,285,3,0.00
0027250175,114,1,0.00
0032909491,178,1,134.73


In [206]:
df_1=df[(df['proxy_metric_new']>0)]

RFM_data_1 = df_1.groupby(['customerid']).agg({'date_col': lambda x: (max_date - x.max()).days,
                                           'callid': 'count',
                                           'proxy_metric_new': 'sum'})

RFM_data_1.rename(columns = {'date_col': 'Recency',
                           'callid': 'Frequency',
                           'proxy_metric_new': 'MonetaryValue'},
                inplace=True)
Monetary_Qs_df=RFM_data_1.describe()
Monetary_min=Monetary_Qs_df.loc["min","MonetaryValue"]
Monetary_25=Monetary_Qs_df.loc["25%","MonetaryValue"]
Monetary_50=Monetary_Qs_df.loc["50%","MonetaryValue"]
Monetary_75=Monetary_Qs_df.loc["75%","MonetaryValue"]
Monetary_max=Monetary_Qs_df.loc["max","MonetaryValue"]

#### Next, we will perform descretization with respect to quantiles. They will be rank 1 to 5, with rank 1 as the highest and rank 5 as the lowest

In [209]:
r_labels = range(5,0,-1)
r_quartiles = pd.qcut(RFM_data['Recency'], 5, labels = r_labels )
RFM_data = RFM_data.assign(R = r_quartiles.values)

In [210]:
RFM_data['F']= np.where((RFM_data['Frequency']<=1),1,
                        np.where((RFM_data['Frequency']==2),2,
                               np.where((RFM_data['Frequency']==3),3,
                                       np.where((RFM_data['Frequency']==4),4,5))))

In [211]:
RFM_data['M']=np.where((RFM_data['MonetaryValue']==0),1,
                      np.where((RFM_data['MonetaryValue']>=Monetary_min) & (RFM_data['MonetaryValue']<Monetary_25),2,
                          np.where((RFM_data['MonetaryValue']>=Monetary_25) & (RFM_data['MonetaryValue']<Monetary_50),3,
                             np.where((RFM_data['MonetaryValue']>=Monetary_50) & (RFM_data['MonetaryValue']<Monetary_75),4
                                               ,5))))

In [212]:
RFM_data.head()

,Recency,Frequency,MonetaryValue,R,F,M
customerid,,,,,,
0003503118,234,3,0.00,1,3,1
0014521825,159,1,0.00,3,1,1
0022804213,285,3,0.00,1,3,1
0027250175,114,1,0.00,3,1,1
0032909491,178,1,134.73,2,1,4


In [213]:
rfm_end_time=dt.datetime.now()
print('Time Taken for only RFM creation: ',rfm_end_time-rfm_start_time)

Time Taken for only RFM creation:  0:02:12.065847


In [214]:
## add clumpiness part here
t1=dt.datetime.now()
clumpiness=pd.pivot_table(df, values='callid', index='customerid', columns='date_col', aggfunc='count')
clumpiness.fillna(0,inplace=True)
result=[]
for i in range(len(clumpiness)):
    #print(runstest_1samp(list(xyz.values[i]), correction = False)[1])
    result.append(runstest_1samp(list(clumpiness.values[i]), correction = False)[1])

clumpiness["result"]=result
clumpiness.reset_index(drop = False, inplace=True,level=0)
clumpiness=clumpiness[['customerid','result']]
clumpiness.columns.name=None
clumpiness.rename(columns={'result':'Clumpiness'},inplace=True)
clumpiness['C']=clumpiness['Clumpiness'].apply(lambda x: 1 if x>0.9 
                                               else (2 if x>0.8 
                                                     else (3 if x>0.1 
                                                           else (4 if x>0.0005 
                                                                 else 5))))
t2=dt.datetime.now()
print("Time Taken for Clumpiness creation: ",t2-t1)

Time Taken for Clumpiness creation:  0:05:31.489241


In [215]:
RFMC_data=pd.merge(RFM_data,clumpiness, on='customerid', how='left')

#### Making RFMC_segment by concatenate RFMC and RFM_Score by adding RFMC

In [216]:
def join_rfm(x):return (str(x['R']) + str(x['F']) + str(x['M']) + str(x['C']))
RFMC_data['RFMC_Segment'] = RFMC_data.apply(join_rfm, axis=1)
RFMC_data['RFMC_Score'] = RFMC_data[['R','F','M','C']].sum(axis=1).astype(int)
Loyalty_Level = ['Wood','Iron', 'Bronze','Silver','Gold','Platinum']
Score_cuts = pd.qcut(RFMC_data.RFMC_Score, q = 6, labels = Loyalty_Level)
RFMC_data['RFMC_Loyalty_Level'] = Score_cuts.values
RFMC_data.reset_index(drop=True, inplace=True)
RFMC_data=RFMC_data[['customerid', 'Recency', 'Frequency', 'MonetaryValue', 'Clumpiness','R', 'F', 'M', 'C',
       'RFMC_Segment', 'RFMC_Score', 'RFMC_Loyalty_Level']]
print("Time taken to create complete RFMC: ", dt.datetime.now()-rfm_start_time)

In [220]:
RFMC_data[["RFMC_Loyalty_Level", "Recency", "Frequency", "MonetaryValue","Clumpiness", "RFMC_Score"]].groupby("RFMC_Loyalty_Level").agg(
    ["mean","min","max", "count"])

Recency                   Frequency                    \
                          mean  min  max   count      mean min   max   count   
RFMC_Loyalty_Level                                                             
Wood                236.395569  164  310  233562  1.065803   1     2  233562   
Iron                158.779479  111  310  138762  1.165528   1     3  138762   
Bronze              116.744511   54  310  151674  1.208579   1     4  151674   
Silver               79.022661    2  310  158203  1.248364   1    21  158203   
Gold                104.428689    2  310  108559  1.910841   1    22  108559   
Platinum             83.059030    1  310  106742  3.324933   1  2342  106742   

                   MonetaryValue                        Clumpiness  \
                            mean  min       max   count       mean   
RFMC_Loyalty_Level                                                   
Wood                    0.887749  0.0     75.06  233562   0.935565   
Iron                    6.420657  0.0    124.91  138762   0.934073   
Bronze                 14.222283  0.0    201.28  151674   0.932739   
Silver                 30.489015  0.0   1200.06  158203   0.931650   
Gold                   77.187181  0.0   1366.78  108559   0.882160   
Platinum              147.505623  0.0  32534.83  106742   0.596001   

                                                   RFMC_Score                  
                             min       max   count       mean min max   count  
RFMC_Loyalty_Level                                                             
Wood                9.355651e-01  0.935565  233562   4.553301   4   5  233562  
Iron                8.946841e-01  0.935565  138762   6.000000   6   6  138762  
Bronze              8.501109e-01  0.935565  151674   7.000000   7   7  151674  
Silver              8.051444e-01  0.935565  158203   8.000000   8   8  158203  
Gold                3.304995e-37  0.935565  108559   9.454223   9  10  108559  
Platinum            1.788495e-57  0.945493  106742  12.745424  11  20  106742

In [240]:
RFMC_data.set_index('customerid',inplace=True)

In [241]:
RFMC_data.head()

,Recency,Frequency,MonetaryValue,Clumpiness,R,F,M,C,RFMC_Segment,RFMC_Score,RFMC_Loyalty_Level
customerid,,,,,,,,,,,
0003503118,234,3,0.00,0.935565,1,3,1,1,1311,6,Iron
0014521825,159,1,0.00,0.935565,3,1,1,1,3111,6,Iron
0022804213,285,3,0.00,0.935565,1,3,1,1,1311,6,Iron
0027250175,114,1,0.00,0.935565,3,1,1,1,3111,6,Iron
0032909491,178,1,134.73,0.935565,2,1,4,1,2141,8,Silver


## To MySQL

connection = pymysql.connect(host='10.75.102.17', user='sfr_ai_blue_user', password='SfR_116#611$', 
                             port=3307, db = 'smexplorerdata')
cursor=connection.cursor()

In [242]:
engine = create_engine("mysql+pymysql://{user}:{password}@IP_here:port_here/{db}"
                       .format(user='user_here', password='password_here', 
                             db = 'db_here'))


In [243]:
t=text("delete from schema.`rfmc_lookup`")
engine.execute(t)

In [244]:
RFMC_data.to_sql('rfmc_lookup', con=engine, if_exists='append')
print("Data Transferred to MySQL AI Blue Server")

Data Transferred to MySQL AI Blue Server


## Thank You